In [77]:
#utils 
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
def  symbol_to_path(symbol,base_dir='/home/machine_learning/anaconda3/ml4trading/data/'):
    """returns  CSV  file path given symbol"""
    return os.path.join(base_dir,"{}.csv".format(str(symbol)))
def get_data(symbols,dates):
    """Reads stock  data( adjusted  close ) for  given symbols  from  csv  files   and returns  entire  df"""
    df=pd.DataFrame(index=dates)
    if 'SPY' not in symbols :
        symbols.insert(0,'SPY')
    for symbol in symbols :
        #read and join data  for  each  symbol 
        df_temp=pd.read_csv(symbol_to_path(symbol),index_col="Date",parse_dates=True,usecols=['Date','Adj Close'],na_values=['nan'])
        df_temp=df_temp.rename(columns={'Adj Close':symbol})
        df=df.join(df_temp,how='inner')
        if symbol == 'PSY':
            df = df.dropna(subset=["SPY"])
    return  df
def normalize_data(df):
    """Normalize stock prices using the first row of the dataframe."""
    return df / df.ix[0]
def plot_data(df,title="Stock prices"):
    import  matplotlib
    from  matplotlib import pyplot as plt
    """ plot  stock prices"""
    %matplotlib inline
    ax=df.plot(title=title, fontsize=2)
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    plt.show()
def plot_selected(df, columns,  start_index,end_index):
    subseted=df.ix[pd.date_range(start_index,end_index),columns].dropna()
    plot_data(subseted)
        

In [66]:
def compute_daily_returns(df):
    #(price[t]/price[t-1])-1    to  give percent
    daily_returns=df.copy()
    daily_returns[1:]=(daily_returns[1:]/daily_returns.values[:-1])-1
    daily_returns.ix[0,:]=0
    return  daily_returns
    

In [67]:
a=np.array([1,2,3,4])

In [68]:
b=a*3

In [69]:
d=pd.DataFrame({'a':a,'b':b})

In [70]:
d

,a,b
0,1,3
1,2,6
2,3,9
3,4,12


In [71]:
compute_daily_returns(d)

,a,b
0,0.000000,0.000000
1,1.000000,1.000000
2,0.500000,0.500000
3,0.333333,0.333333


In [73]:
#looks ok

In [138]:
def  get_statistics(historical_prices,alloc=[0.5,0.5],risk_free_rate=0.0,period=252.0,total_invest=1):
    #1  normalize prices  :
    historical_prices=normalize_data(historical_prices)
    #2 for  each  row in  historical normalized data   multiplies  with allocations  to  apply percentage of investment
    allocated_history=historical_prices*alloc
    #3  ending   values  of  those allocations
    end_val=allocated_history.sum(axis=1)
    #4 cummulative return : (s[t]/s[0])-1
    end_val=list(end_val)#because  simming on row    brings   series
    cumul=(end_val[-1]/end_val[0])-1
    #5 daily returns - must  convert back to    df
    end_val=pd.DataFrame(end_val)
    daily_return=compute_daily_returns(end_val)
    #6  and 7  average daily return 
    average_daily_return=daily_return.mean()
    std_daily_return=daily_return.std()
    #8  sharpe  ratio    average  of returns - risk  rate  divided  by   std of   returns  *  sqrt period
    sharpe_ratio=(daily_return -risk_free_rate).mean()/std_daily_return*np.sqrt(period)
    #9 compute ending value of portofolio 
    # to do
    return  sharpe_ratio
    
    

In [90]:
d

,a,b
0,1,3
1,2,6
2,3,9
3,4,12


In [106]:
alloc=normalize_data(d)*[0.5,0.5]#d * allocations
alloc

,a,b
0,0.5,0.5
1,1.0,1.0
2,1.5,1.5
3,2.0,2.0


In [107]:
end=alloc.sum(axis=1)#end of   the end of the  day  is sum by rows   meaning  how  much  current day  bringed
end

0    1.0
1    2.0
2    3.0
3    4.0
dtype: float64

In [114]:
list(end)[-1]

4.0

In [136]:
get_statistics(d)

0    17.461959
dtype: float64

In [118]:
pd.DataFrame(end)

,0
0,1.0
1,2.0
2,3.0
3,4.0


In [120]:
compute_daily_returns(pd.DataFrame(end))

,0
0,0.000000
1,1.000000
2,0.500000
3,0.333333
